In [1]:
%matplotlib inline
from PIL import Image
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
import os
import itertools
from tqdm import *
from tqdm import tqdm_notebook as tqdm
import random
import numpy as np
import math
import matplotlib.pyplot as plt

from IPython.display import clear_output

# Step 1: Load Data Set

In [2]:
fileNames = []
for file in os.listdir("data/train_1"):
    if file.endswith(".jpg"):
        fileNames.append(file)
# fileNamesDF = pd.DataFrame(fileNames)

In [22]:
#only take certain artist so that the probabilities aren't basically all false when doing the tupples
minNumPaintingsPerArtist = 50

# #for each artist, only take n of their paintings
numPaintingsPerAuthor = 100


trainInfo = pd.read_csv('data/train_info.csv')
trainInfo = trainInfo[trainInfo['filename'].isin(fileNames)]

mostPopularArtists = trainInfo['artist'].value_counts()[trainInfo['artist'].value_counts() > minNumPaintingsPerArtist]
# print(mostPopularArtists.head())
# fileNamesDFAll = trainInfo[trainInfo['artist'].isin(mostPopularArtists.index)]
# print(fileNamesDFAll.head())



fileNamesDF = pd.DataFrame()
for artist in tqdm(mostPopularArtists.index):
    fileNamesDF = fileNamesDF.append(trainInfo[trainInfo['artist'] == artist][:numPaintingsPerAuthor])

In [23]:
# Load the all_data_info for the dataset we are using
allInfo = pd.read_csv('data/all_data_info.csv')
allInfo = allInfo[allInfo['new_filename'].isin(fileNamesDF['filename'])]

featuresDF = pd.DataFrame(allInfo['new_filename'])
featuresDF['pixelsx'] = allInfo['pixelsx']
featuresDF['pixelsy'] = allInfo['pixelsy']

In [24]:
def normalizeSeries(series, maxNum):
    series = series/maxNum
    return series

In [29]:
def featurizeImage(filename):
    # Extract the features from the actual image
    jpgfile = Image.open("data/train_1/" + filename)
#     print(jpgfile)
    
    bands = jpgfile.getbands()
    if bands[0] == 'L':
        # grayscale image
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['grayscale'])
#         imageDF['grayscale'] = normalizeSeries(imageDF['grayscale',])
    elif len(bands) == 4:
        #get red, green and blue chanels 
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['red','green','blue','alpha'])
        imageDF['red'] = normalizeSeries(imageDF['red'],255)
        imageDF['green'] = normalizeSeries(imageDF['green'],255)
        imageDF['blue'] = normalizeSeries(imageDF['blue'],255)  
        imageDF['alpha'] = normalizeSeries(imageDF['alpha'],255)  
    elif len(bands) == 3:
        #get red, green and blue chanels 
        imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['red','green','blue'])
        imageDF['red'] = normalizeSeries(imageDF['red'],255)
        imageDF['green'] = normalizeSeries(imageDF['green'],255)
        imageDF['blue'] = normalizeSeries(imageDF['blue'],255)  
    else:
        imageDF = pd.DataFrame(list(jpgfile.getdata()))
        print(bands)
        return imageDF, bands

    return imageDF

In [30]:
featuresDF.head()

,new_filename,pixelsx,pixelsy
201,101495.jpg,4728.0,3528.0
602,11889.jpg,3893.0,3072.0
697,14108.jpg,3899.0,2874.0
773,1155.jpg,3899.0,2732.0
784,18290.jpg,3911.0,2700.0


In [ ]:
imageFeaturesDF = pd.DataFrame(index=featuresDF.index,columns=['features']) 
print(imageFeaturesDF.index)
for row in tqdm(list(featuresDF.index)):
    imageFeatures = featurizeImage(featuresDF.loc[row]['new_filename'])
    imageFeaturesDF.loc[row]['features'] = imageFeatures



Int64Index([  201,   602,   697,   773,   784,   829,   854,   867,   879,
              891,
            ...
            81867, 82039, 82905, 84014, 90393, 92193, 93066, 93403, 96412,
            97634],
           dtype='int64', length=1276)


In [ ]:
imageFeaturesDF.head()

In [ ]:
imageFeaturesDF.to_csv('trainImgsAllcolorBands.csv')